# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for AAPL",
"context": "The task involves using statistical modeling and machine learning to identify trends and predict market movements for the selected stock AAPL. The Data Analyst needs to gather, analyze, and interpret market data to provide insights and alerts about significant market opportunities or threats for AAPL."
}


> Entering new CrewAgentExecutor chain...
I need to start by gathering the latest market data for 

Final Answer: The latest market data for AAPL from Yahoo Finance shows that as of 9:45 AM EST, Apple Inc. (AAPL) is trading at $227.05, marking a 0.91% increase. The market cap is at 3.432T, with a 1-year return of 19.70% and a 5-year return of 241.74%. Additionally, key events, performance overview, financial highlights, and company insights are provided for further analysis.

> Finished chain.
 

The latest market data for AAPL from Yahoo Finance shows that as of 9:45 AM EST, Apple Inc. (AAPL) is trading at $227.05, marking a 0.91% increase. The market cap is at 3.432T, with a 1-year return of 19.70% and a 5-year return of 241.74%. Additionally, key events, performance overview, financial highlights, and company insights are provided for further analysis.

Final Answer:
The Data Analyst has been delegated the task of continuously monitoring and analyzing market data for AAPL. The latest market data for AAPL from Yahoo Finance shows that as of 9:45 AM EST, Apple Inc. (AAPL) is trading

Final Answer: 
After analyzing the latest market data for AAPL from Yahoo Finance, here are the key insights:

- AAPL is currently trading at $227.06, showing a 0.92% increase.
- The market cap for AAPL stands at 3.432T.
- AAPL has a 1-year return of 20.29% and a 5-year return of 252.51%.
- Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide.
- The company offers various subscription-based services such as Apple Arcade, Apple Fitness+, Apple Music, Apple News+, and Apple TV+.
- AAPL has a Profit Margin of 23.97% and Return on Equity of 157.41%.
- The company has a Trailing P/E of 37.01 and a Forward P/E of 30.30.

Based on this data, AAPL continues to show strong performance in the market with consistent growth and a diversified product and service portfolio.

> Finished chain.
 

After analyzing the latest market data for AAPL from Yahoo Finance, here are the key insights:

- AAPL is currently trading at $227.06,

Final Answer: Page Not Found - Unable to retrieve information on approved trading strategies for AAPL.

> Finished chain.
 

Page Not Found - Unable to retrieve information on approved trading strategies for AAPL.

Final Answer: Unfortunately, I was unable to delegate the task to the Trading Strategy Developer as the information was not found. I will need to reassess and determine the best course of action to complete the task.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: Unfortunately, I was unable to delegate the task to the Trading Strategy Developer as the information was not found. I will need to reassess and determine the best course of action to complete the task.
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Evaluate the risks associated with the proposed trading strategies and execution plans for AAPL. Provide a detailed analysis of potential risks and suggest mitigation strategies.


> Entering new CrewAgentExecutor chain...
I need to gather as muc

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Unfortunately, due to technical difficulties, I was unable to gather the necessary information to conduct a comprehensive risk analysis report for AAPL. As the Crew Manager, I will work on resolving these issues and finding alternative ways to obtain the required data to complete the task successfully. Thank you for your understanding.